<a href="https://colab.research.google.com/github/gusmaomarcos/machine-learning-avancada/blob/main/Word2Vec_pt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!python -m spacy download pt_core_news_sm

In [ ]:
import pandas as pd
import spacy
import pt_core_news_sm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Sempre checar se o caminho está correto.
teste = "/content/drive/MyDrive/Databases/word2vec_2/teste.csv"
treino = "/content/drive/MyDrive/Databases/word2vec_2/treino.csv"

dados_teste = pd.read_csv(teste)
dados_treino = pd.read_csv(treino)

In [ ]:
dados_treino.sample(3)

,title,text,date,category,subcategory,link
37973,Reajustes salariais devem voltar a ganhar da i...,"Depois de dois anos no negativo, os reajustes ...",2017-01-24,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/01/1...
38999,Governo estuda mudar lei para facilitar fusão ...,A Oi enviou a um grupo de trabalho criado no M...,2015-07-11,mercado,NaN,http://www1.folha.uol.com.br/mercado/2015/11/1...
56506,Trump busca alternativa para resolver disputa ...,"A administração do presidente americano, Donal...",2017-02-28,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/02/1...


In [ ]:
nlp = spacy.load("pt_core_news_sm")
#nlp = pt_core_news_sm.load()
#doc = nlp("Esta é uma frase.")
#print([(w.text, w.pos_) for w in doc])

Exploração

In [ ]:
texto = 'Rio de Janeiro 123 @@ é uma## cidade maravilhosa'
doc = nlp(texto)

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
doc[0]

Rio

In [ ]:
type(doc[0])

spacy.tokens.token.Token

In [ ]:
doc.ents

(Rio de Janeiro, @@)

In [ ]:
doc[0].is_stop

False

In [ ]:
doc[1].is_stop

True

Tratamento

In [ ]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino['title'])

In [ ]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)
    if len(tokens_validos) > 2:
        return ' '.join(tokens_validos)

In [ ]:
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [ ]:
type(nlp)

spacy.lang.pt.Portuguese

In [ ]:
from time import time
t0=time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size=1000, n_process=-1)]
t1=time()
print((t1-t0)/60)

4.920016213258108


In [ ]:
#ps.: Célula anterior demorando, em média, 5 min na minha máquina.

In [ ]:
titulos_tratados = pd.DataFrame({'titulo':textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [ ]:
from gensim.models import Word2Vec

#Quando sg=0, treino modelo com arquitetura cbow. Quando sg=1 treino com skipgrams.
#Parametro window é o range de contexto, para frente e para trás.

'''w2v_modelo = Word2Vec(sg=0,
                      window=2,
                      vector_size=300,
                      min_count=5,
                      alpha=0.03,
                      min_alpha=0.0007)'''

'w2v_modelo = Word2Vec(sg=0,\n                      window=2,\n                      vector_size=300,\n                      min_count=5,\n                      alpha=0.03,\n                      min_alpha=0.0007)'

In [ ]:
#w2v_modelo

In [ ]:
print(len(titulos_tratados))

90000


In [ ]:
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

84466


In [ ]:
lista_lista_tokens = [titulo.split(' ') for titulo in titulos_tratados.titulo]

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    def __init__(self):
       self.epoch = 0

    def on_epoch_end(self, model):
       loss = model.get_latest_training_loss()
       if self.epoch == 0:
           print('Loss após a época {}: {}'.format(self.epoch, loss))
       else:
           print('Loss após a época {}: {}'.format(self.epoch, loss- self.loss_previous_step))
       self.epoch += 1
       self.loss_previous_step = loss

In [ ]:
import logging
logging.getLogger().setLevel(logging.DEBUG)
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.INFO)

w2v_modelo = Word2Vec(sg=0,
                      window=2,
                      vector_size=300,
                      min_count=5,
                      alpha=0.03,
                      min_alpha=0.0007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

DEBUG:gensim.utils:starting a new internal lifecycle event log for Word2Vec
INFO:gensim.utils:Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2024-07-10T12:09:44.978796', 'gensim': '4.3.2', 'python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]', 'platform': 'Linux-6.1.85+-x86_64-with-glibc2.35', 'event': 'created'}
INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
INFO:gensim.models.word2vec:PROGRESS: at

In [ ]:
dir(w2v_modelo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 'corpus_count',
 

In [ ]:
#rerun

In [ ]:
dir(w2v_modelo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 'corpus_count',
 

In [ ]:
w2v_modelo.corpus_count

84466

In [ ]:
w2v_modelo.train(lista_lista_tokens,
                 total_examples=w2v_modelo.corpus_count,
                 epochs=30,
                 callbacks=[callback()])

INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'training model with 3 workers on 12924 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2024-07-10T13:11:59.951961', 'gensim': '4.3.2', 'python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]', 'platform': 'Linux-6.1.85+-x86_64-with-glibc2.35', 'event': 'train'}
INFO:gensim.models.word2vec:EPOCH 0 - PROGRESS: at 88.82% examples, 422676 words/s, in_qsize 5, out_qsize 0
DEBUG:gensim.models.word2vec:job loop exiting, total 55 jobs
DEBUG:gensim.models.word2vec:worker exiting, processed 21 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 2 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 17 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 1 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 17 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 0 more thre

Loss após a época 0: 0.0


INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 87.00% examples, 421271 words/s, in_qsize 5, out_qsize 0
DEBUG:gensim.models.word2vec:job loop exiting, total 55 jobs
DEBUG:gensim.models.word2vec:worker exiting, processed 20 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 2 more threads
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 1 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 18 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.word2vec:EPOCH 1: training on 540242 raw words (486272 effective words) took 1.1s, 428349 effective words/s
DEBUG:gensim.models.word2vec:worker exiting, processed 17 jobs


Loss após a época 1: 0.0


INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 85.14% examples, 403660 words/s, in_qsize 5, out_qsize 0
DEBUG:gensim.models.word2vec:job loop exiting, total 55 jobs
DEBUG:gensim.models.word2vec:worker exiting, processed 18 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 2 more threads
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 1 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 19 jobs
DEBUG:gensim.models.word2vec:worker exiting, processed 18 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.word2vec:EPOCH 2: training on 540242 raw words (486216 effective words) took 1.2s, 414622 effective words/s


...

Loss após a época 28: 0.0


INFO:gensim.models.word2vec:EPOCH 29 - PROGRESS: at 68.51% examples, 330170 words/s, in_qsize 5, out_qsize 0
DEBUG:gensim.models.word2vec:job loop exiting, total 55 jobs
DEBUG:gensim.models.word2vec:worker exiting, processed 19 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 2 more threads
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 1 more threads
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.word2vec:EPOCH 29: training on 540242 raw words (486315 effective words) took 1.3s, 366902 effective words/s
INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'training on 16207260 raw words (14584856 effective words) took 39.6s, 368349 effective words/s', 'datetime': '2024-07-10T13:12:39.548583', 'gensim': '4.3.2', 'python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]', 'platform': 'Linux-6.1.85+-x86_64-with-glibc2.35', 'event': 'train'}


Loss após a época 29: 0.0


DEBUG:gensim.models.word2vec:worker exiting, processed 19 jobs
DEBUG:gensim.models.word2vec:worker exiting, processed 17 jobs


(14584856, 16207260)

In [ ]:
w2v_modelo.wv.most_similar('rio')

[('sp', 0.3533506393432617),
 ('campinas', 0.3391982316970825),
 ('rn', 0.332835853099823),
 ('bahia', 0.33153626322746277),
 ('bh', 0.3189869821071625),
 ('ilhabela', 0.3182435929775238),
 ('cuiabá', 0.3027421236038208),
 ('rocinha', 0.3026760518550873),
 ('janeiro', 0.3025020360946655),
 ('manaus', 0.3007639944553375)]

In [ ]:
w2v_modelo.wv.most_similar('ferrari')

[('mercedes', 0.48027363419532776),
 ('vettel', 0.42429837584495544),
 ('nasr', 0.4209877550601959),
 ('schumacher', 0.410127729177475),
 ('mclaren', 0.40768393874168396),
 ('grid', 0.4008604884147644),
 ('raikkonen', 0.3805752396583557),
 ('rosberg', 0.37927085161209106),
 ('áustria', 0.37195563316345215),
 ('pontuar', 0.36714446544647217)]

Modelo Skipgram

In [ ]:
w2v_modelo_sg = Word2Vec(sg=1, #skipgram on
                      window=5, #janela maior
                      vector_size=300,
                      min_count=5,
                      alpha=0.03,
                      min_alpha=0.0007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

w2v_modelo_sg.train(lista_lista_tokens,
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs=30,
                 callbacks=[callback()])

DEBUG:gensim.utils:starting a new internal lifecycle event log for Word2Vec
INFO:gensim.utils:Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2024-07-10T13:15:43.181483', 'gensim': '4.3.2', 'python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]', 'platform': 'Linux-6.1.85+-x86_64-with-glibc2.35', 'event': 'created'}
INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
INFO:gensim.models.word2vec:PROGRESS: at

Loss após a época 0: 0.0


INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 31.50% examples, 150643 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 66.67% examples, 150776 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 88.85% examples, 132886 words/s, in_qsize 5, out_qsize 0
DEBUG:gensim.models.word2vec:job loop exiting, total 55 jobs
DEBUG:gensim.models.word2vec:worker exiting, processed 19 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 2 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 17 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 1 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 19 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.word2vec:EPOCH 1: training on 540242 raw words (486033 effective words) took 3.8s, 129505 effective words/s


Loss após a época 1: 0.0


INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 18.54% examples, 82802 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 42.66% examples, 97715 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 61.16% examples, 94928 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 90.67% examples, 104772 words/s, in_qsize 4, out_qsize 1
DEBUG:gensim.models.word2vec:job loop exiting, total 55 jobs
DEBUG:gensim.models.word2vec:worker exiting, processed 18 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 2 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 19 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 1 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 18 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.word2vec:EPOCH 2: training on 540242 raw 

...

INFO:gensim.models.word2vec:EPOCH 28 - PROGRESS: at 25.95% examples, 123063 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 28 - PROGRESS: at 62.98% examples, 148352 words/s, in_qsize 5, out_qsize 0
DEBUG:gensim.models.word2vec:job loop exiting, total 55 jobs
INFO:gensim.models.word2vec:EPOCH 28 - PROGRESS: at 92.52% examples, 143362 words/s, in_qsize 5, out_qsize 0
DEBUG:gensim.models.word2vec:worker exiting, processed 19 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 2 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 17 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 1 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 19 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.word2vec:EPOCH 28: training on 540242 raw words (486168 effective words) took 3.3s, 148095 effective words/s


Loss após a época 28: 0.0


INFO:gensim.models.word2vec:EPOCH 29 - PROGRESS: at 25.95% examples, 122156 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 29 - PROGRESS: at 64.83% examples, 148520 words/s, in_qsize 5, out_qsize 0
DEBUG:gensim.models.word2vec:job loop exiting, total 55 jobs
INFO:gensim.models.word2vec:EPOCH 29 - PROGRESS: at 94.36% examples, 146357 words/s, in_qsize 4, out_qsize 0
DEBUG:gensim.models.word2vec:worker exiting, processed 19 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 2 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 18 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 1 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 18 jobs
DEBUG:gensim.models.word2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.word2vec:EPOCH 29: training on 540242 raw words (486053 effective words) took 3.3s, 147585 effective words/s
INFO:gensim.utils:Word2Vec lifecyc

Loss após a época 29: 0.0


(14584530, 16207260)

In [ ]:
w2v_modelo_sg.wv.most_similar('rio')

[('janeiro', 0.4253093898296356),
 ('upps', 0.39934101700782776),
 ('upp', 0.3912830352783203),
 ('deodoro', 0.39108407497406006),
 ('olympics', 0.38744398951530457),
 ('public', 0.38372156023979187),
 ('in', 0.38265272974967957),
 ('leblon', 0.37723174691200256),
 ('are', 0.3766883611679077),
 ('security', 0.37441807985305786)]

In [ ]:
w2v_modelo_sg.wv.most_similar('ferrari')

[('vettel', 0.6336820125579834),
 ('jerez', 0.5736679434776306),
 ('raikkonen', 0.57235187292099),
 ('mônaco', 0.49907374382019043),
 ('sauber', 0.48776552081108093),
 ('schumacher', 0.4865109920501709),
 ('nasr', 0.4861512780189514),
 ('mercedes', 0.483132541179657),
 ('verstappen', 0.4663412570953369),
 ('pontuar', 0.4542502164840698)]

In [ ]:
w2v_modelo.wv.save_word2vec_format('/content/drive/MyDrive/Databases/word2vec_2/modelo_cbow.txt', binary=False)
w2v_modelo_sg.wv.save_word2vec_format('/content/drive/MyDrive/Databases/word2vec_2/modelo_skipgram.txt', binary=False)

INFO:gensim.models.keyedvectors:storing 12924x300 projection weights into /content/drive/MyDrive/Databases/word2vec_2/modelo_cbow.txt
DEBUG:smart_open.smart_open_lib:{'uri': '/content/drive/MyDrive/Databases/word2vec_2/modelo_cbow.txt', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.models.keyedvectors:storing 12924x300 projection weights into /content/drive/MyDrive/Databases/word2vec_2/modelo_skipgram.txt
DEBUG:smart_open.smart_open_lib:{'uri': '/content/drive/MyDrive/Databases/word2vec_2/modelo_skipgram.txt', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
